# Tests with Neural Networks

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [31]:
df = pd.read_csv("data/shared_all_records.csv")
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.set_index('Time', inplace=True)
df['Energy'] = df['Energy'].astype(np.float32)
df

,Energy,Location
Time,,
2020-10-06 00:15:00,0.076,PT41CV
2020-10-06 00:30:00,0.072,PT41CV
2020-10-06 00:45:00,0.072,PT41CV
2020-10-06 01:00:00,0.068,PT41CV
2020-10-06 01:15:00,0.032,PT41CV
...,...,...
2022-11-25 00:00:00,0.764,PT87ZW
2022-11-25 00:15:00,0.660,PT87ZW
2022-11-25 00:30:00,0.692,PT87ZW


In [ ]:

# Split data into training and validation sets
split_fraction = 0.8
train_split = int(split_fraction * len(df))
train_data = df[:train_split]
val_data = df[train_split:]

# Normalize the data
train_mean = train_data.mean()
train_std = train_data.std()
train_data = (train_data - train_mean) / train_std
val_data = (val_data - train_mean) / train_std

# Prepare the data for training
def create_dataset(data):
    X = []
    y = []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:i+window_size])
        y.append(data.iloc[i+window_size])
    return np.array(X), np.array(y)

window_size = 7 # number of days to use as input
train_X, train_y = create_dataset(train_data)
val_X, val_y = create_dataset(val_data)

# Define the model
model = keras.Sequential([
    layers.LSTM(64, input_shape=(window_size, 1)),
    layers.Dense(1)
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='mse',
              metrics=['mae'])

# Train the model
history = model.fit(train_X, train_y, epochs=100, batch_size=64, validation_data=(val_X, val_y))

# Evaluate the model
test_data = pd.read_csv('energy_consumption_test.csv')
test_data['date'] = pd.to_datetime(test_data['date'])
test_data = test_data.set_index('date')
test_data = (test_data - train_mean) / train_std
test_X, test_y = create_dataset(test_data)
test_loss, test_mae = model.evaluate(test_X, test_y)

# Make predictions with the model
predictions = model.predict(test_X)

# Plot the predictions against the actual values
import matplotlib.pyplot as plt

plt.plot(test_data.index[window_size:], predictions, label='Predictions')
plt.plot(test_data.index[window_size:], test_y, label='Actual')
plt.legend()
plt.show()